# Further Data Extraction
### Imports

In [9]:
from dotenv import load_dotenv
from os import environ as ENV

import psycopg2
import pandas as pd
import altair as alt

### Connect to CSV file

In [10]:
load_dotenv()

conn = psycopg2.connect(user=ENV["DB_USER"],
                        password=ENV["DB_PASSWORD"],
                        host=ENV["DB_HOST"],
                        port=ENV["DB_PORT"],
                        dbname=ENV["DB_NAME"])

In [13]:
# All exhibition interactions

with conn.cursor() as cursor:
    tag_album_stmt = (
        """
        SELECT * FROM tag
        JOIN album_tag_assignment USING (tag_id)
        JOIN album USING (album_id)
        JOIN sale_album_assignment USING (album_id)
        JOIN sale USING (sale_id)
        JOIN country USING (country_id);
        """
    )
    tag_track_stmt = (
        """
        SELECT * FROM tag
        JOIN track_tag_assignment USING (tag_id)
        JOIN track USING (track_id)
        JOIN sale_track_assignment USING (track_id)
        JOIN sale USING (sale_id)
        JOIN country USING (country_id);
        """
    )
    album_stmt = (
        """
        SELECT * FROM sale
        JOIN sale_album_assignment USING (sale_id)
        JOIN country USING (country_id)
        JOIN album USING (album_id)
        JOIN artist_album_assignment USING (album_id)
        JOIN artist USING (artist_id)
        JOIN album_tag_assignment USING (album_id)
        JOIN tag USING (tag_id);
        """
    )
    album_sales = pd.read_sql_query(album_stmt, conn)
    track_stmt = (
        """
        SELECT * FROM sale
        JOIN sale_track_assignment USING (sale_id)
        JOIN country USING (country_id)
        """
    )
    album_sales = pd.read_sql_query(album_stmt, conn)
    merch_stmt = (
        """
        SELECT * FROM sale
        JOIN sale_merchandise_assignment USING (sale_id)
        JOIN country USING (country_id)
        """
    )
    tag_album_sales = pd.read_sql_query(tag_album_stmt, conn)
    tag_track_sales = pd.read_sql_query(tag_track_stmt, conn)
    album_sales = pd.read_sql_query(album_stmt, conn)
    merch_sales = pd.read_sql_query(merch_stmt, conn)
    track_sales = pd.read_sql_query(track_stmt, conn)
tag_track_sales.head(5)

/var/folders/ld/chk5sydx4z3fjl57zcfkqt0m0000gn/T/ipykernel_24946/2146081391.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  album_sales = pd.read_sql_query(album_stmt, conn)
/var/folders/ld/chk5sydx4z3fjl57zcfkqt0m0000gn/T/ipykernel_24946/2146081391.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  album_sales = pd.read_sql_query(album_stmt, conn)
/var/folders/ld/chk5sydx4z3fjl57zcfkqt0m0000gn/T/ipykernel_24946/2146081391.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tag_album_sales = pd.read_sql_query(tag_al

,country_id,sale_id,track_id,tag_id,tag_name,track_tag_assignment_id,track_name,release_date,art_url,url,track_assignment_id,sold_for,utc_date,country_name
0,1,2,1,15,alternative,1,Birdhouse In Your Soul,2020-05-01,https://f4.bcbits.com/img/a2830032266_7.jpg,https://tmbg.bandcamp.com/track/birdhouse-in-y...,1,1.00,2025-06-17 07:49:02.636310,United States
1,6,11,2,74,electronic,2,Mental Tekno,2024-06-10,https://f4.bcbits.com/img/a2342602636_7.jpg,https://belektek420.bandcamp.com/track/mental-...,2,1.73,2025-06-17 07:49:39.016579,Israel
2,7,12,3,74,electronic,3,Magic Touch,2024-10-16,https://f4.bcbits.com/img/a3550571989_7.jpg,https://tomar1.bandcamp.com/track/magic-touch,3,0.98,2025-06-17 07:49:46.829427,Germany
3,7,12,3,106,house,4,Magic Touch,2024-10-16,https://f4.bcbits.com/img/a3550571989_7.jpg,https://tomar1.bandcamp.com/track/magic-touch,3,0.98,2025-06-17 07:49:46.829427,Germany
4,7,12,3,139,naarm,5,Magic Touch,2024-10-16,https://f4.bcbits.com/img/a3550571989_7.jpg,https://tomar1.bandcamp.com/track/magic-touch,3,0.98,2025-06-17 07:49:46.829427,Germany


In [18]:
tag_album_sales_cleaned = tag_album_sales.drop(['country_id',
                                                'album_tag_assignment_id', 'album_assignment_id'], axis=1)
tag_track_sales_cleaned = tag_track_sales.drop(['country_id',
                                                'track_tag_assignment_id', 'track_assignment_id'], axis=1)

In [19]:
tag_album_sales_cleaned

,sale_id,album_id,tag_id,tag_name,album_name,release_date,art_url,url,sold_for,is_physical,utc_date,country_name
0,1,1,65,drum and bass,Trying EP,None,https://f4.bcbits.com/img/a3214254574_7.jpg,https://repertoire.bandcamp.com/album/trying-ep,6.78,False,2025-06-17 07:49:01.897336,United States
1,1,1,66,drum&bass,Trying EP,None,https://f4.bcbits.com/img/a3214254574_7.jpg,https://repertoire.bandcamp.com/album/trying-ep,6.78,False,2025-06-17 07:49:01.897336,United States
2,1,1,74,electronic,Trying EP,None,https://f4.bcbits.com/img/a3214254574_7.jpg,https://repertoire.bandcamp.com/album/trying-ep,6.78,False,2025-06-17 07:49:01.897336,United States
3,1,1,122,jungle,Trying EP,None,https://f4.bcbits.com/img/a3214254574_7.jpg,https://repertoire.bandcamp.com/album/trying-ep,6.78,False,2025-06-17 07:49:01.897336,United States
4,1,1,127,lmajor,Trying EP,None,https://f4.bcbits.com/img/a3214254574_7.jpg,https://repertoire.bandcamp.com/album/trying-ep,6.78,False,2025-06-17 07:49:01.897336,United States
...,...,...,...,...,...,...,...,...,...,...,...,...
325,1248,1111,5360,['classical',Alba,2024-12-13,https://f4.bcbits.com/img/a0192592220_7.jpg,https://guillaumeponcelet.bandcamp.com/album/alba,1.16,False,2025-06-16 08:42:59.598284,France
326,1333,1111,5360,['classical',Alba,2024-12-13,https://f4.bcbits.com/img/a0192592220_7.jpg,https://guillaumeponcelet.bandcamp.com/album/alba,1.16,False,2025-06-16 08:42:59.598284,France
327,1249,1139,5360,['classical',Haven,2022-04-22,https://f4.bcbits.com/img/a0170663135_7.jpg,https://guillaumeponcelet.bandcamp.com/album/h...,5.79,False,2025-06-16 08:42:59.859811,France
328,1334,1139,5360,['classical',Haven,2022-04-22,https://f4.bcbits.com/img/a0170663135_7.jpg,https://guillaumeponcelet.bandcamp.com/album/h...,5.79,False,2025-06-16 08:42:59.859811,France


In [ ]:
# Step 1: Drop duplicate (sale_id, tag_id) to avoid overcounting


,tag_id,tag_name,sale_count,total_revenue,top_country_1,top_country_2,top_country_3
151,5222,['electronic',46,292.06,Germany,Belgium,United States
54,74,electronic,16,167.35,United Kingdom,Czech Republic,United States
156,5275,['afro',12,86.00,Germany,NaN,NaN
157,5284,['african',12,96.00,Germany,NaN,NaN
155,5253,['alternative',6,76.60,Belgium,Germany,Italy
...,...,...,...,...,...,...,...
50,68,dub techno,1,3.99,Czech Republic,NaN,NaN
49,67,dub,1,2.71,United Kingdom,NaN,NaN
46,64,drum & bass,1,2.71,United Kingdom,NaN,NaN
43,61,doom,1,1.00,United States,NaN,NaN


In [22]:
tag_track_sales_cleaned_unique = tag_track_sales_cleaned.drop_duplicates(
    subset=['sale_id', 'tag_id'])

popular_track_tags = (
    tag_track_sales_cleaned_unique
    .groupby(['tag_id', 'tag_name'])
    .agg(sale_count=('sale_id', 'nunique'))
    .reset_index()
)

popular_track_tags = popular_track_tags.sort_values(
    by='sale_count', ascending=False)
print(popular_track_tags)

     tag_id            tag_name  sale_count
27       74          electronic          28
79      194              techno          12
44      106               house          11
26       73             electro           9
100    5222       ['electronic'           8
..      ...                 ...         ...
61      149      outre-acidcore           1
60      148          outre-acid           1
59      143            nu-disco           1
58      142               noise           1
55      137  monkeytown records           1

[111 rows x 3 columns]


In [26]:
tag_album_money_cleaned = tag_album_sales_cleaned.drop_duplicates(
    subset=['sale_id', 'tag_id'])

tag_revenue = (
    tag_album_money_cleaned
    .groupby(['tag_id', 'tag_name'])
    .agg(total_revenue=('sold_for', 'sum'))
    .reset_index()
)

tag_country_revenue = (
    tag_album_money_cleaned
    .groupby(['tag_id', 'country_name'])
    .agg(country_revenue=('sold_for', 'sum'))
    .reset_index()
)

tag_country_revenue['rank'] = tag_country_revenue.groupby('tag_id')['country_revenue'] \
    .rank(method='first', ascending=False)

top_countries_pivot = (
    tag_country_revenue[tag_country_revenue['rank'] <= 3]
    .sort_values(['tag_id', 'rank'])
    .pivot(index='tag_id', columns='rank', values='country_name')
    .rename(columns={1.0: 'top_country_1', 2.0: 'top_country_2', 3.0: 'top_country_3'})
    .reset_index()
)

final_df = tag_revenue.merge(top_countries_pivot, on='tag_id', how='left')
final_df = final_df.sort_values(by='total_revenue', ascending=False)

print(final_df)

     tag_id           tag_name  total_revenue   top_country_1  top_country_2  \
151    5222      ['electronic'         292.06         Germany  United States   
54       74         electronic         167.35  United Kingdom  United States   
157    5284         ['african'          96.00         Germany            NaN   
156    5275            ['afro'          86.00         Germany            NaN   
155    5253     ['alternative'          76.60         Belgium          Italy   
..      ...                ...            ...             ...            ...   
80      115       instrumental           1.00   United States            NaN   
81      116  instrumental rock           1.00   United States            NaN   
129     186        stoner rock           1.00   United States            NaN   
128     185       stoner metal           1.00   United States            NaN   
123     178       sludge metal           1.00   United States            NaN   

    top_country_3  
151       Belgium  

In [119]:
album_sales_cleaned = album_sales.drop(['country_id', 'album_id', 'album_assignment_id',
                               'tag_id', 'artist_album_assignment_id', 'album_tag_assignment_id'], axis=1)

In [120]:
album_sales_cleaned.head(2) #including tags

,artist_id,sale_id,utc_date,sold_for,country_name,album_name,release_date,art_url,url,artist_name,tag_name
0,35,1,2025-06-16 14:57:01.303047,27.99,United States,"""Obscure Power"" vinyl LP (4 color options!) [SHIPPING FROM USA]",2025-04-01,https://f4.bcbits.com/img/0039178913_37.jpg,https://questmaster.bandcamp.com/album/obscure-power,Quest Master,ambient
1,35,1,2025-06-16 14:57:01.303047,27.99,United States,"""Obscure Power"" vinyl LP (4 color options!) [SHIPPING FROM USA]",2025-04-01,https://f4.bcbits.com/img/0039178913_37.jpg,https://questmaster.bandcamp.com/album/obscure-power,Quest Master,maine


In [121]:
album_sales_cleaned_no_tags = album_sales_cleaned.drop(
    columns=['tag_name']).drop_duplicates()

In [122]:
album_sales_cleaned_no_tags.head(5)

,artist_id,sale_id,utc_date,sold_for,country_name,album_name,release_date,art_url,url,artist_name
0,35,1,2025-06-16 14:57:01.303047,27.99,United States,"""Obscure Power"" vinyl LP (4 color options!) [SHIPPING FROM USA]",2025-04-01,https://f4.bcbits.com/img/0039178913_37.jpg,https://questmaster.bandcamp.com/album/obscure-power,Quest Master
14,36,2,2025-06-16 14:57:04.052832,16.32,United Kingdom,TRIP MAPS 1,2025-05-16,https://f4.bcbits.com/img/0039690726_37.jpg,https://theaa.bandcamp.com/album/trip-maps-1,FSOLdigital
18,37,3,2025-06-16 14:57:04.969738,11.57,United Kingdom,The Cartel Remixes,2014-04-28,https://f4.bcbits.com/img/a3431917184_7.jpg,https://theaa.bandcamp.com/album/the-cartel-remixes,The Amorphous Androgynous
22,38,4,2025-06-16 14:57:09.762950,25.00,United States,VINYL,2021-07-21,https://f4.bcbits.com/img/0034961914_37.jpg,https://samgendelsamwilkes.bandcamp.com/album/music-for-saxofone-and-bass-guitar-more-songs,Sam Gendel & Sam Wilkes
23,39,5,2025-06-16 14:57:11.259960,9.28,United States,Maara - Goddess Within,2022-04-12,https://f4.bcbits.com/img/a1642426852_7.jpg,https://radiant-records.bandcamp.com/album/maara-goddess-within,Maara


In [123]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

album_sales_cleaned_no_tags.groupby(['album_name', 'artist_name', 'url']).size().reset_index(
    name='count').sort_values('count', ascending=False)

,album_name,artist_name,url,count
178,"Limited Edition 12"" vinyl BLUE MARBLE",BoriRock,https://daupe.bandcamp.com/album/house-fire,8
3,(Listen for) the Rag and Bone Man - reissue,And Also The Trees,https://andalsothetrees.bandcamp.com/album/listen-for-the-rag-and-bone-man-reissue,2
9,"7"" vinyl with vintage action figure",Rave Wars,https://ravewars.bandcamp.com/album/rave-wars-iv-the-acid-awakens,2
183,Limited Edition Clear Gold Vinyl LP,Sloan,https://sloanmusic.bandcamp.com/album/based-on-the-best-seller,2
38,Bizzy B Origination E.P ( Limited Blue Vinyl Release ),Bizzy B,https://thebizzyb.bandcamp.com/album/bizzy-b-origination-e-p-digital,2
...,...,...,...,...
129,Guardianista Class,Hellfish & Bryan Fury,https://hkv017hellfish.bandcamp.com/album/guardianista-class,1
128,Grime Chronicles,Numa Crew,https://numacrew1.bandcamp.com/album/grime-chronicles,1
127,Greatest Hits,Remo Drive,https://remodrive.bandcamp.com/album/greatest-hits,1
126,Gravity Racer,Hiccup Heart,https://hiccupheart.bandcamp.com/album/gravity-racer,1
